In [14]:
import pandas as pd
import numpy as np

datafile="d:/data/original_data.xls"
outputfile="d:/data/output.xls"

#import data#
data=pd.read_excel(datafile,encoding="utf-8")

#delete useless variables#
column_name=['time', 'condition','heating','warning','actual tem','volumn','water volumn','remain time','tem']
data=data[column_name]

#event
threshold=pd.Timedelta(minutes=5)#experience threshold
data['time']=pd.to_datetime(data['time'],format='%Y%m%d%H%M%S')
data=data[data['water volumn']>0]

# find the best threshold time#
n=4

def event_num(ts):
    d=data["time"].diff()>ts
    return d.sum()+1

dt=[pd.Timedelta(minutes=i) for i in np.arange(1,9,0.25)]
h=pd.DataFrame(dt,columns=['threshold'])
h['event']=h['threshold'].apply(event_num)
h['slope']=h['event'].diff()/0.25
h['slope index']=pd.rolling_mean(h['slope'].abs(),n)
ts=h['threshold'][h['slope index'].idxmin()-n]

if ts > threshold:
    ts=pd.Timedata(mintues=4)
print (ts)

#divide the event
threshold_op=pd.Timedelta(minutes=4)
d=data['time'].diff()>threshold_op
data['event_index']=d.cumsum()+1
print(data)






0 days 00:04:00
                   time condition heating warning actual tem volumn  \
2   2014-10-19 07:01:56         关       关       关       30°C     0%   
56  2014-10-19 07:38:16         关       关       关       30°C     0%   
381 2014-10-19 09:46:38         关       关       关       29°C     0%   
382 2014-10-19 09:46:40         关       关       关       29°C     0%   
384 2014-10-19 09:47:15         关       关       关       29°C     0%   

     water volumn remain time   tem  event_index  
2               8         0分钟  50°C            1  
56              8         0分钟  50°C            2  
381            16         0分钟  50°C            3  
382            13         0分钟  50°C            3  
384            20         0分钟  50°C            3  


D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:30: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()


In [31]:
#create system#
data_group=data.groupby('event_index')
result=pd.DataFrame()

for n,g in data_group:
    df=pd.Timedelta(seconds=2)
    temp = pd.DataFrame(index = [0])
    tstart=g['time'].min()
    tend=g['time'].max()
    temp['the time of event(min)']=(df+tend-tstart).total_seconds()/60
    temp['pause']=(pd.rolling_sum(g['condition']==u'关', 2) == 1).sum()
    temp['total water volum'] = g['water volumn'].sum()
    tdiff = g['time'].diff()
    if len(g['time']) == 1:
        temp['the total time of event(min)'] = dt.total_seconds()/60
    else:
        temp['the total time of event(min)'] = (tdiff.sum() - tdiff.iloc[1]/2 - tdiff.iloc[len(tdiff)-1]/2).total_seconds()/60
    temp['avg of water volum'] = temp['total water volum']/temp['the total time of event(min)']
    temp['use water time rate']=temp['the time of event(min)']/temp['the total time of event(min)']
    result = result.append(temp, ignore_index = True)
    

print(result)



D:\pythonanaconda\lib\site-packages\ipykernel\__main__.py:10: FutureWarning: pd.rolling_sum is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).sum()


     the time of event(min)  pause  total water volum  \
0                  0.033333      0                  8   
1                  0.033333      0                  8   
2                  0.650000      0                 49   
3                  0.033333      0                 22   
4                  0.033333      0                  8   
5                 14.150000      1               7282   
6                  0.266667      0                384   
7                 11.283333      0               3050   
8                  8.750000      0               3876   
9                  9.300000      0               4040   
10                 0.033333      0                  8   
11                 0.166667      0                 16   
12                 0.033333      0                  8   
13                15.933333      0               3895   
14                 0.066667      0                 23   
15                 0.500000      0                496   
16                 4.133333    

In [33]:
import keras
from keras.models import Sequential#initialize neural network
from keras.layers import Dense#create the level of neural network

testdatafile='d:/data/test_neural_network_data.xls'
traindatafile='d:/data/train_neural_network_data.xls'
traindata=pd.read_excel(traindatafile,encoding='utf-8')
testdata=pd.read_excel(testdatafile,encoding='utf-8')

X = traindata.iloc[:,5:17].values
y = traindata.iloc[:, 4].values

X_test = testdata.iloc[:,5:17].values
y_test = testdata.iloc[:, 4].values


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 17, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) 



# Adding the second hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
#second layer would not need the input_dim

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
#output_dim 出来的结果的数量

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#optimizer=最优的算法；loss:loss function(要选择) 

# Fitting the ANN to the Training set
classifier.fit(X, y, batch_size = 10, epochs = 100)

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

Epoch 1/100
28/28 [==============================] - 1s - loss: 0.7000 - acc: 0.3929     
Epoch 2/100
28/28 [==============================] - 0s - loss: 0.6665 - acc: 0.8214     
Epoch 3/100
28/28 [==============================] - 0s - loss: 0.6406 - acc: 0.7857     
Epoch 4/100
28/28 [==============================] - 0s - loss: 0.6193 - acc: 0.7857     
Epoch 5/100
28/28 [==============================] - 0s - loss: 0.5934 - acc: 0.7857     
Epoch 6/100
28/28 [==============================] - 0s - loss: 0.5564 - acc: 0.7857     

In [34]:
cm

array([[ 1,  3],
       [ 0, 17]])